<a href="https://colab.research.google.com/github/ShaymaFathima/FOXIOM/blob/main/Automated_attendance_report_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade fireworks-ai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain-fireworks --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import sqlite3
from langchain_fireworks import Fireworks

In [ ]:
# File paths for biometry and PMS data
biometry_file_path = '/content/Biometry.csv'
pms_file_path = '/content/pms_new.csv'

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('attendance_data.db')
cursor = conn.cursor() #creates a cursor object that allows to execute SQL commands

In [ ]:
# Load each CSV file
biometry_df = pd.read_csv(biometry_file_path)
biometry_df = biometry_df.rename(columns={' InTime': 'InTime',' OutTime': 'OutTime'})

pms_df = pd.read_csv(pms_file_path)

# Insert data directly, replacing existing tables if they exist
biometry_df.to_sql('biometry', conn, if_exists='replace', index=False)
pms_df.to_sql('pms', conn, if_exists='replace', index=False)

961

In [ ]:
# Commit the changes
conn.commit() #commits the changes to the database

In [ ]:
query1 = """
SELECT
    b.`E_Code`,
    p.employee_name,
    p.date as pms_date,
    b.Status as biometry_status,
    b.InTime,
    b.OutTime,
    b.`Tot_Dur` as biometry_duration,
    p.hours_worked as pms_status
FROM
    pms p
LEFT JOIN
    biometry b
ON
    b.Name = p.employee_name
    AND b.Attendance_Date = p.date
"""

In [ ]:
cursor.execute(query1)
data1 = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
monthly_data_df = pd.DataFrame(data1, columns=columns)

In [ ]:
# Identify holidays (dates where InTime is 'None' for all employees)
holidays = monthly_data_df.groupby('pms_date').apply(lambda x: x['InTime'].isnull().all()).reset_index()
holidays = holidays[holidays[0] == True]['pms_date'].tolist()

In [ ]:
def parse_biometry_duration(duration_str):
    try:
        hours, minutes = map(int, duration_str.split(':'))
        return hours * 60 + minutes
    except:
        return 0

def parse_pms_duration(duration_str):
    try:
        parts = duration_str.split(' ')
        hours = 0
        minutes = 0
        for part in parts:
            if 'h' in part:
                hours = int(part.replace('h', ''))
            elif 'm' in part:
                minutes = int(part.replace('m', ''))
        return hours * 60 + minutes
    except:
        return 0

In [ ]:
def generate_prompt(employee_name, employee_code, employee_data, holidays):
    total_working_days = len(employee_data[~employee_data['pms_date'].isin(holidays)]['pms_date'].unique())
    present_days = employee_data[employee_data['biometry_status'].str.contains('Present', na=False)].shape[0]
    leave_days = total_working_days - present_days

    biometry_total_duration = employee_data['biometry_duration'].apply(parse_biometry_duration).sum()
    pms_total_duration = employee_data['pms_status'].apply(parse_pms_duration).sum()
    prompt = f"""
    You are an AI language model assistant. Your task is to evaluate an employee's attendance and performance based on the provided data and collect specific details about each employee to determine their attendance record and productivity.
    Employee Name: {employee_name}
    Employee Code: {employee_code}
    Total Working Days of the Month: {total_working_days}
    Total Days Present: {present_days}
    Total Days Absent/On Leave: {leave_days}
    Biometry Total Duration: {biometry_total_duration}
    PMS Total Duration: {pms_total_duration}

    Additional Notes:
    - 'Present (No OutPunch)' in biometry status indicates the employee forgot to punch out.
    - The majority InTime of an employee may indicate their actual working hours.
    - For senior employees, some flexibility in InTime is allowed.
    - All other employees are expected to work from 9:00 AM to 6:00 PM with a 1-hour break.

    Provide your insights on the given data and suggest any improvements for the employee's attendance and performance.
    """
    return prompt

In [ ]:
# Get unique employee names and codes
employee_info = monthly_data_df[['employee_name', 'E_Code']].drop_duplicates()

# Initialize Fireworks LLM
llm = Fireworks(
    api_key="yB0Gk2lfUCbEHOmjZYddJCbuci3Lt8PyVGqquH1PA441GQ08",
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=512,
    temperature=0.7,  # Adjust for more creative or conservative responses
    top_p=0.9
)

# Generate and store reports for each employee
employee_reports = {}

for _, row in employee_info.iterrows():
    employee_name = row['employee_name']
    employee_code = row['E_Code']
    employee_data = monthly_data_df[monthly_data_df['employee_name'] == employee_name]
    prompt = generate_prompt(employee_name, employee_code, employee_data, holidays)

    # Get the response from the LLM
    response = llm.generate([prompt])

    # Extract the generated text
    report_text = response.generations[0][0].text

    # Store the report in the dictionary
    employee_reports[employee_name] = report_text

# Print the reports in a structured format
for employee_name, report in employee_reports.items():
    print(f"Report for {employee_name}:\n")
    print(report)
    print("\n" + "="*80 + "\n")

Report for Foxiom Accounts:



Based on the provided data, Foxiom Accounts has been absent for all working days of the month, totaling 24 days. This is a significant concern, as consistent absences can negatively impact productivity and workload distribution within the team or department.

Attendance Improvements:
1. Encourage the employee to follow a consistent attendance schedule, adhering to the standard working hours of 9:00 AM to 6:00 PM with a 1-hour break.
2. Address the issue of not punching out in the biometry system, which may indicate a lack of clarity regarding the process or potential forgetfulness.
3. Schedule regular check-ins with the employee to discuss attendance patterns and offer support in addressing any underlying issues that may be contributing to their absences.

Performance Improvements:
1. Although there is no direct measure of productivity in the given data, the employee's extended absence may still impact overall team performance. It's essential to monitor w

In [ ]:
# Example of generating a report for a specific employee
employee_name = 'Vishnu Sarath KT'  # Replace with the desired employee name
employee_code = 'FXL1006'  # Replace with the corresponding employee code

# Filter the employee's data
employee_data = monthly_data_df[monthly_data_df['employee_name'] == employee_name]

# Generate the prompt for the LLM
prompt = generate_prompt(employee_name, employee_code, employee_data, holidays)

# Get the response from the LLM
response = llm.generate([prompt])

# Assuming the response has the desired text output
report_text = response.generations[0][0].text

print(f"Report for {employee_name}:\n")
print(report_text)

Report for Vishnu Sarath KT:

1) The employee's total working days of the month are 24. They were present for 21 days and absent or on leave for 0 days. This is an excellent attendance record.
    
    2) The employee's total duration of biometry is 8440 minutes, which is equivalent to 140.67 hours. This is slightly above the expected working hours of 126 hours (16 hours x 24 days - 12 hours break).
    
    3) The employee's total duration of PMS is 10094 minutes, which is equivalent to 168.23 hours. This is significantly above the expected working hours.
    
    4) Based on the given data, the employee seems to be working beyond their expected hours and is present on all working days. However, there is a discrepancy in their InTime for the PMS system, which indicates they are working beyond their expected working hours, but they are not following the standard work hours.

    Suggestions:
    - The employee's attendance and performance are excellent, and they are working beyond thei

In [ ]:
import pandas as pd
import sqlite3
import os
from langchain_fireworks import Fireworks

# File paths for biometry and PMS data
biometry_file_path = '/content/Biometry.csv'
pms_file_path = '/content/pms_new.csv'

# Connect to SQLite database using context manager
with sqlite3.connect('attendance_data.db') as conn:
    # Load each CSV file
    biometry_df = pd.read_csv(biometry_file_path)
    biometry_df = biometry_df.rename(columns={' InTime': 'InTime', ' OutTime': 'OutTime'})

    pms_df = pd.read_csv(pms_file_path)

    # Insert data directly, replacing existing tables if they exist
    biometry_df.to_sql('biometry', conn, if_exists='replace', index=False)
    pms_df.to_sql('pms', conn, if_exists='replace', index=False)

    query1 = """
    SELECT
        b.E_Code,
        p.employee_name,
        p.date as pms_date,
        b.Status as biometry_status,
        b.InTime,
        b.OutTime,
        b.Tot_Dur as biometry_duration,
        p.hours_worked as pms_status
    FROM
        pms p
    LEFT JOIN
        biometry b
    ON
        b.Name = p.employee_name
        AND b.Attendance_Date = p.date
    """
    monthly_data_df = pd.read_sql_query(query1, conn)

# Identify holidays (dates where InTime is 'None' for all employees)
holidays = monthly_data_df.groupby('pms_date')['InTime'].apply(lambda x: x.isnull().all()).reset_index()
holidays = holidays[holidays['InTime'] == True]['pms_date'].tolist()

def parse_duration(duration_str, pattern):
    if not isinstance(duration_str, str):
        return 0
    try:
        parts = duration_str.split(pattern)
        hours, minutes = 0, 0
        for part in parts:
            if 'h' in part:
                hours = int(part.replace('h', ''))
            elif 'm' in part:
                minutes = int(part.replace('m', ''))
        return hours * 60 + minutes
    except:
        return 0

def generate_prompt(employee_name, employee_code, employee_data, holidays):
    total_working_days = len(employee_data[~employee_data['pms_date'].isin(holidays)]['pms_date'].unique())
    present_days = employee_data[employee_data['biometry_status'].str.contains('Present', na=False)].shape[0]
    leave_days = total_working_days - present_days

    biometry_total_duration = employee_data['biometry_duration'].apply(lambda x: parse_duration(x, ':')).sum()
    pms_total_duration = employee_data['pms_status'].apply(lambda x: parse_duration(x, ' ')).sum()

    prompt = f"""
    You are an AI language model assistant. Your task is to evaluate an employee's attendance and performance based on the provided data and collect specific details about each employee to determine their attendance record and productivity.
    Employee Name: {employee_name}
    Employee Code: {employee_code}
    Total Working Days of the Month: {total_working_days}
    Total Days Present: {present_days}
    Total Days Absent/On Leave: {leave_days}
    Biometry Total Duration: {biometry_total_duration}
    PMS Total Duration: {pms_total_duration}

    Additional Notes:
    - 'Present (No OutPunch)' in biometry status indicates the employee forgot to punch out.
    - The majority InTime of an employee may indicate their actual working hours.
    - For senior employees, some flexibility in InTime is allowed.
    - All other employees are expected to work from 9:00 AM to 6:00 PM with a 1-hour break.

    Provide your insights on the given data and suggest any improvements for the employee's attendance and performance.
    """
    return prompt

# Get unique employee names and codes
employee_info = monthly_data_df[['employee_name', 'E_Code']].drop_duplicates()

# Initialize Fireworks LLM with environment variables for sensitive data
fireworks_api_key = os.getenv("FIREWORKS_API_KEY", "yB0Gk2lfUCbEHOmjZYddJCbuci3Lt8PyVGqquH1PA441GQ08")
llm = Fireworks(
    api_key=fireworks_api_key,
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=512,
    temperature=0.7,
    top_p=0.9
)

# Generate and store reports for each employee
employee_reports = {}

for _, row in employee_info.iterrows():
    employee_name = row['employee_name']
    employee_code = row['E_Code']
    employee_data = monthly_data_df[monthly_data_df['employee_name'] == employee_name]
    prompt = generate_prompt(employee_name, employee_code, employee_data, holidays)

    # Get the response from the LLM
    response = llm.generate([prompt])

    # Extract the generated text
    report_text = response.generations[0][0].text

    # Store the report in the dictionary
    employee_reports[employee_name] = report_text

# Print the reports in a structured format
for employee_name, report in employee_reports.items():
    print(f"Report for {employee_name}:\n")
    print(report)
    print("\n" + "="*80 + "\n")


Report for Foxiom Accounts:


    Based on the provided data, it appears that the employee named 'Foxiom Accounts' has not been present at work during the entire month. This is a significant concern, as it indicates a lack of commitment and poor attendance. The employee has not clocked in using biometric data, and there is no information available about their productivity.

    To address this issue, the following steps could be taken:

    1. Schedule a meeting with the employee to discuss their attendance and performance.
    2. Establish clear expectations for attendance and work hours, and ensure that the employee understands the company's attendance policy.
    3. Implement a performance improvement plan to address any issues related to productivity and work quality.
    4. Monitor the employee's attendance and performance closely to ensure that improvements are made.
    5. Consider offering additional support or resources to help the employee improve their attendance and perform